In [1]:
# third party
import sys
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path
import psyplot.project as psy
import pandas as pd
import xarray
from netCDF4 import Dataset,date2num
import metpy.calc as calc
from metpy.units import units
import datetime as dt
import pandas as pd
from iconarray.plot import formatoptions # import plotting formatoptions (for use with psyplot)
import iconarray as iconvis # import self-written modules from iconarray
import io, os, sys, types
import pickle

# first party
sys.path.append('../utilities_tlezuo/')
from timefunctions import *
import varfunctions as vf
import locfunctions as lf


INFO:numexpr.utils:Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
# define class of pc_dict
class Point_coordinates:
    def __init__(self, shortname, name, color, marker, lat, lon, altitude, iconID, iconHSURF, iconHHL, iconHFL, height_dict):
        self.short = shortname
        self.name = name
        self.color = color
        self.marker = marker
        self.lat = lat
        self.lon = lon
        self.alt = altitude
        self.iconID = iconID 
        self.iconHSURF = iconHSURF
        self.iconHHL = iconHHL
        self.iconHFL = iconHFL
        self.hdict = height_dict 


# load pc_dict and its lists
pc_dict = np.load('../utilities_tlezuo/pc_dict.npy',allow_pickle=True).item()
with open ('../utilities_tlezuo/pc_iconID_list','rb') as f:
    pc_iconID_list = pickle.load(f)
with open ('../utilities_tlezuo/pc_short_list','rb') as a:
    pc_short_list = pickle.load(a)

In [3]:
###############################################################################################
## DECIDE ##

# RUN
run = 'RUN2_reference'
filepath = '/store/s83/tlezuo/'+run+'/out_hfsu/'
savepath = '/users/tlezuo/icon-vis/data/data_hfsu/'

# VARIABLES
pvars_list= [vf.T, vf.TKE, vf.TKEtend_shear, vf.TKEtend_conv, vf.TKEVELtend] # 3d
spvars_list = [vf.T_2M,vf.QV_2M,vf.U_10M,vf.V_10M, vf.SH_s, vf.LH_s, vf.SWnet_s, vf.LWnet_s] #2d surf
scvars_list = [vf.VEL_10M, vf.DIR_10M] # to be calculated

# TIME
startdate = dt.datetime(2019,9,13,00,00)
enddate = dt.datetime(2019,9,14,00,00)
plotfreq = '0h0min10s'
simdate = dt.datetime(2019,9,12,12,00) # no change,. simulation start
plotdates = pd.date_range(startdate,enddate,freq=plotfreq)
# lfffhfsu00120


In [4]:
## BIG READ IN ##

# read in all nc files at once, parallelized only on node!
# data=xarray.open_mfdataset(filepath+'lfffhfsu00120*')

# read in all nc files at once, parallelized only on node!
data=xarray.open_mfdataset(filepath+'lfffhfsu00120*')

# htd timeseries = selecting our locations
hfsu_htd_data = data.sel(ncells=pc_iconID_list,drop=False)

# calculate new vars
hfsu_htd_data = hfsu_htd_data.assign(VEL_10m=vf.calculate_wind_vel_from_uv(hfsu_htd_data['u_10m'],hfsu_htd_data['v_10m']))
hfsu_htd_data = hfsu_htd_data.assign(DIR_10m=vf.calculate_wind_dir_from_uv(hfsu_htd_data['u_10m'],hfsu_htd_data['v_10m']))

# surface timeseries
hfsu_ts_data = hfsu_htd_data.sel(height_4=80)

In [5]:
# how to select a loaction
# hfsu_ts_data.sel(ncells=pc_dict['kols'].subsetID).T

In [6]:
## SAVE ##
# hfsu_htd_data.to_netcdf(savepath+'hfsu_htd_data_'+run+'.nc')
hfsu_ts_data.to_netcdf(savepath+'hfsu_ts_data_'+run+'.nc')
# intsu_htd_data.to_netcdf(savepath+'intsu_htd_data_'+run+'.nc')

In [7]:
# test if it worked
run = 'RUN4_cosmolev'
data_loaded=xarray.open_mfdataset(savepath+'hfsu_ts_data_'+run+'.nc')
data_loaded

<xarray.Dataset>
Dimensions:        (time: 8641, ncells: 19, vertices: 3, height: 1, height_2: 1, height_3: 4, bnds: 2)
Coordinates:
  * time           (time) datetime64[ns] 2019-09-13 ... 2019-09-14
    clon           (ncells) float32 dask.array<chunksize=(19,), meta=np.ndarray>
    clat           (ncells) float32 dask.array<chunksize=(19,), meta=np.ndarray>
  * height         (height) float64 2.0
  * height_2       (height_2) float64 10.0
  * height_3       (height_3) float64 77.0 78.0 79.0 80.0
    height_4       float64 ...
Dimensions without coordinates: ncells, vertices, bnds
Data variables: (12/17)
    clon_bnds      (time, ncells, vertices) float32 dask.array<chunksize=(8641, 19, 3), meta=np.ndarray>
    clat_bnds      (time, ncells, vertices) float32 dask.array<chunksize=(8641, 19, 3), meta=np.ndarray>
    height_3_bnds  (time, height_3, bnds) float64 dask.array<chunksize=(8641, 4, 2), meta=np.ndarray>
    T_2M           (time, height, ncells) float64 dask.array<chunksize=(8641, 1, 19), meta=np.ndarray>
    u_10m          (time, height_2, ncells) float64 dask.array<chunksize=(8641, 1, 19), meta=np.ndarray>
    v_10m          (time, height_2, ncells) float64 dask.array<chunksize=(8641, 1, 19), meta=np.ndarray>
    ...             ...
    ddt_tke_hsh    (time, ncells) float64 dask.array<chunksize=(8641, 19), meta=np.ndarray>
    DTKE_CON       (time, ncells) float64 dask.array<chunksize=(8641, 19), meta=np.ndarray>
    ddt_tke        (time, ncells) float64 dask.array<chunksize=(8641, 19), meta=np.ndarray>
    TKE            (time, ncells) float64 dask.array<chunksize=(8641, 19), meta=np.ndarray>
    VEL_10m        (time, height_2, ncells) float64 dask.array<chunksize=(8641, 1, 19), meta=np.ndarray>
    DIR_10m        (time, height_2, ncells) float64 dask.array<chunksize=(8641, 1, 19), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.4 (http://mpimet...
    Conventions:          CF-1.6
    number_of_grid_used:  1
    uuidOfHGrid:          655488b8-6e60-ac09-a653-9b1ce37a2b20
    uuidOfVGrid:          6686a69e-1192-fdad-0fef-84a3fb642c80
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    title:                ICON simulation
    source:               git@gitlab.dkrz.de:icon/icon-nwp.git@820171bea7dab6...
    history:              /oprusers/osm/opr.aare/abs/icon at 20230207 082433
    references:           see MPIM/DWD publications
    comment:              Tobia Lezuo, MCH (tlezuo) on nid003048 (Linux 5.3.1...